## Extract Machine Learning Features

Compute bootstrap connectivity matrices using selected ROIs:
- Motor (M1, premotor, SMA)
- Basal ganglia (Putamen, Caudate, GP, NAcc)
- Executive control (DLPFC, IFJ, IFS)



In [ ]:
# Libraries
import numpy as np
import pandas as pd
from pathlib import Path
from mne_connectivity import spectral_connectivity_epochs
from numpy.random import default_rng

# ───── CONFIG ─────
PROJECT_BASE = Path("/home/directory")
DATA_DIR = PROJECT_BASE / "derivatives/phase_epochs"
ROI_FILE = Path("/gt_atlas/roi_labels_enhanced.csv")  # Use enhanced file
OUTPUT_BASE = PROJECT_BASE / "derivatives/ml_features"

# Frequency bands
BANDS = {
    "Theta": (4, 8),
    "Alpha": (8, 12),
    "Low_Beta": (13, 20),
    "High_Beta": (20, 30),
    "Low_Gamma": (30, 60),
    "High_Gamma": (60, 120)
}

SFREQ = 500.0
METHOD = 'wpli2_debiased'
N_BOOTSTRAPS = 50
CHUNK_SIZE = 20  # Reduced to match lower epoch counts
RNG = default_rng(seed=42)

# ───── LOAD ROI METADATA AND DEFINE TARGET ROIs ─────
roi_df = pd.read_csv(ROI_FILE)
FULL_ROI_NAMES = roi_df['roi_name'].tolist()
N_FULL = len(FULL_ROI_NAMES)

# Motor: M1, premotor, supplementary, eye
motor_rois = roi_df[
    (roi_df['functional_system'] == 'Motor') |
    (roi_df['sub_system'].isin(['Primary', 'Premotor', 'Supplementary', 'Eye']))
]['roi_name'].tolist()

# Basal ganglia
basal_rois = roi_df[
    (roi_df['functional_system'] == 'BasalGanglia') |
    (roi_df['region_full_name'].str.contains(r'Putamen|Caudate|Globus Pallidus|Nucleus Accumbens', case=False, na=False))
]['roi_name'].tolist()

# Executive control
executive_rois = roi_df[
    (roi_df['functional_system'] == 'Frontoparietal') &
    (roi_df['sub_system'].isin(['DLPFC', 'IFJ', 'IFS', 'VLPFC']))
]['roi_name'].tolist()

# Combine and deduplicate
TARGET_ROIS = sorted(set(motor_rois + basal_rois + executive_rois))
TARGET_INDICES = [FULL_ROI_NAMES.index(r) for r in TARGET_ROIS]
N_TARGET = len(TARGET_ROIS)

print(f"✅ Using {N_TARGET} selected ROIs for bootstrap:")
for r in TARGET_ROIS:
    row = roi_df[roi_df['roi_name'] == r].iloc[0]
    print(f"  - {r} | {row['functional_system']} / {row['sub_system']}")

# ───── HELPER FUNCTIONS ─────
def load_subject_epochs(subject_id: str):
    data = np.load(DATA_DIR / f"{subject_id}.npz")
    return {
        'inphase': data['inphase_epochs'],      # (n_epochs, N_FULL, n_times)
        'outphase': data['outphase_epochs']
    }

def compute_connectivity(data, fmin, fmax):
    con = spectral_connectivity_epochs(
        data=data,
        method=METHOD,
        mode='multitaper',
        sfreq=SFREQ,
        fmin=fmin,
        fmax=fmax,
        faverage=True,
        verbose=False,
        n_jobs=1
    )
    mat = con.get_data(output='dense').squeeze()
    mat = (mat + mat.T) / 2
    np.fill_diagonal(mat, 0)
    return mat


def process_subject_bootstrap(subject_id: str):
    print(f"\n🔄 Processing bootstrap: {subject_id} ({N_TARGET} ROIs)")
    epochs_full = load_subject_epochs(subject_id)
    out_dir = OUTPUT_BASE / subject_id
    out_dir.mkdir(parents=True, exist_ok=True)

    # Validate ROI count
    sample_data = epochs_full['inphase'] if epochs_full['inphase'].size > 0 else epochs_full['outphase']
    if sample_data.shape[1] != N_FULL:
        raise ValueError(f"ROI mismatch in {subject_id}: expected {N_FULL}, got {sample_data.shape[1]}")

    for band, (fmin, fmax) in BANDS.items():
        for cond, data_full in epochs_full.items():
            if data_full.size == 0:
                continue

            n_epochs = data_full.shape[0]
            if n_epochs < 5:
                print(f"  ⚠ Skipping {band}/{cond}: only {n_epochs} epochs")
                continue

            # Subselect target ROIs
            data_sub = data_full[:, TARGET_INDICES, :]  # (n_epochs, N_TARGET, n_times)

            # Optional: check if input data is all zero (sanity check)
            if np.allclose(data_sub, 0.0):
                print(f"  ⚠ Warning: {band}/{cond} data is all zero!")
                continue

            boot_list = []
            for i in range(N_BOOTSTRAPS):
                try:
                    idx = RNG.choice(n_epochs, size=CHUNK_SIZE, replace=True)
                    chunk = data_sub[idx]

                    # Skip if chunk is all zero (shouldn't happen, but safe)
                    if np.allclose(chunk, 0.0):
                        continue

                    mat = compute_connectivity(chunk, fmin, fmax)

                    # 🔍 VALIDATION: reject zero or near-zero matrices
                    if np.allclose(mat, 0.0, atol=1e-10):
                        print(f"  ⚠ Zero matrix detected in {subject_id} {band} {cond} (bootstrap {i})")
                        continue

                    # Optional: reject matrices with NaN or Inf
                    if not np.isfinite(mat).all():
                        print(f"  ⚠ Non-finite values in {subject_id} {band} {cond} (bootstrap {i})")
                        continue

                    boot_list.append(mat)

                except Exception as e:
                    continue  # silent fail per bootstrap

            if boot_list:
                boot_tensor = np.stack(boot_list, axis=0)  # (N_BOOT, N_TARGET, N_TARGET)
                fname = out_dir / f"bootstrap_{cond}_{band}.npy"
                np.save(fname, boot_tensor)
                print(f"  ✅ Saved {len(boot_list)} valid bootstraps for {band} {cond} ({mat.shape})")
            else:
                print(f"  ❌ No valid bootstraps for {band} {cond}")

✅ Using 70 selected ROIs for bootstrap:
  - CAU-DA-lh | BasalGanglia / Striatum
  - CAU-DA-rh | BasalGanglia / Striatum
  - CAU-VA-lh | BasalGanglia / Striatum
  - CAU-VA-rh | BasalGanglia / Striatum
  - CAU-body-lh | BasalGanglia / Striatum
  - CAU-body-rh | BasalGanglia / Striatum
  - CAU-tail-lh | BasalGanglia / Striatum
  - CAU-tail-rh | BasalGanglia / Striatum
  - L_1_ROI | Somatosensory / Primary
  - L_2_ROI | Somatosensory / Primary
  - L_3a_ROI | Somatosensory / Primary
  - L_3b_ROI | Somatosensory / Primary
  - L_46_ROI | Frontoparietal / DLPFC
  - L_4_ROI | Motor / Primary
  - L_55b_ROI | Motor / Premotor
  - L_6a_ROI | Motor / Premotor
  - L_6d_ROI | Motor / Premotor
  - L_6ma_ROI | Motor / Supplementary
  - L_6mp_ROI | Motor / Supplementary
  - L_6r_ROI | Motor / Premotor
  - L_6v_ROI | Motor / Premotor
  - L_9-46d_ROI | Frontoparietal / DLPFC
  - L_IFJa_ROI | Frontoparietal / IFJ
  - L_IFJp_ROI | Frontoparietal / IFJ
  - L_IFSa_ROI | Frontoparietal / IFS
  - L_IFSp_ROI | F

In [5]:
# ───── RUN ─────
if __name__ == "__main__":
    subject_ids = [f.stem for f in DATA_DIR.glob("Sbj*.npz")]
    print(f"Found {len(subject_ids)} subjects.")
    for sbj in sorted(subject_ids):
        try:
            process_subject_bootstrap(sbj)
        except Exception as e:
            print(f"💥 Fatal error for {sbj}: {e}")
    print(f"\n🎉 Bootstrap matrices done! ({N_TARGET} ROIs, CHUNK_SIZE={CHUNK_SIZE})")

Found 11 subjects.

🔄 Processing bootstrap: Sbj001 (70 ROIs)
  ✅ Saved 50 valid bootstraps for Theta inphase ((70, 70))
  ✅ Saved 50 valid bootstraps for Theta outphase ((70, 70))
  ✅ Saved 50 valid bootstraps for Alpha inphase ((70, 70))
  ✅ Saved 50 valid bootstraps for Alpha outphase ((70, 70))
  ✅ Saved 50 valid bootstraps for Low_Beta inphase ((70, 70))
  ✅ Saved 50 valid bootstraps for Low_Beta outphase ((70, 70))
  ✅ Saved 50 valid bootstraps for High_Beta inphase ((70, 70))
  ✅ Saved 50 valid bootstraps for High_Beta outphase ((70, 70))
  ✅ Saved 50 valid bootstraps for Low_Gamma inphase ((70, 70))
  ✅ Saved 50 valid bootstraps for Low_Gamma outphase ((70, 70))
  ✅ Saved 50 valid bootstraps for High_Gamma inphase ((70, 70))
  ✅ Saved 50 valid bootstraps for High_Gamma outphase ((70, 70))

🔄 Processing bootstrap: Sbj002 (70 ROIs)
  ✅ Saved 50 valid bootstraps for Theta inphase ((70, 70))
  ✅ Saved 50 valid bootstraps for Theta outphase ((70, 70))
  ✅ Saved 50 valid bootstraps fo